# Preprocessing
This file contains the preprocessing steps taken for the texts and the context scripts.
tool (citation)
@misc{11356/1286,
 title = {The {CLASSLA}-{StanfordNLP} model for lemmatisation of standard Slovenian 1.1},
 author = {Ljube{\v s}i{\'c}, Nikola},
 url = {http://hdl.handle.net/11356/1286},
 note = {Slovenian language resource repository {CLARIN}.{SI}},
 copyright = {Creative Commons - Attribution-{ShareAlike} 4.0 International ({CC} {BY}-{SA} 4.0)},
 year = {2020} }

## Preprocessing the context files:

In [164]:
from nltk import Text

# import text:
file1 = Text('../../data/ButalskipolicajInCefizelj_ID_8.txt')
file2 = Text('../../data/Premraženo Sonce Slovene only _ID_15.txt')
file3 = Text('../../data/Veveriček posebne sorte Special Squirrel_ID_12.txt')

# tokenize text:


In [165]:
file1

<Text: . . / . . / d a...>

In [194]:
import pandas as pd
import numpy as np

# data = pd.read_excel('../../data/imapBook/AllDiscussionDataCODED_USE_THIS_14Feb2020_MH.xls')
# data = data.dropna(axis= 1, how = 'all')
# data = data.dropna(axis= 0, how = 'all')
# print(np.array(data.columns).reshape(-1,1))

# grouping the messages by topic
g_data = data.loc[data['Book ID'].isin([8.0, 12.0, 15.0])].groupby('Book ID')
g_messages = g_data[['Topic', 'Message', 'Book relevance']]
book1 = g_messages.get_group(8.0)
book2 =  g_messages.get_group(12.0)
book3 =  g_messages.get_group(15.0)

print(len(book1),
      len(book2),
      len(book3))

# loading slovene stopwords:
SW = pd.read_csv('sloStopWords.txt', sep = '\n').to_numpy()

from nltk import tokenize

def tokenize_data(book):
    question_tokens = []
    text_tokens = []
    for ind, message  in enumerate( book["Message"]):
#         print(message)
        text_tokens.append(tokenize.casual_tokenize(str(message)))
        question_tokens.append(tokenize.casual_tokenize(str(book['Topic'].iloc[ind]).split('/')[0]))
    return question_tokens,text_tokens

from nltk.metrics.distance import edit_distance as Lev

def levhenshtein_dist(tokens1, tokens2, SW = None):
    d = np.ones(len(tokens1))*np.inf
    for i,t1 in enumerate(tokens1):
        tmp_dist = 0
#         n = 0
        for j in t1:
            if SW is not None:
                if j in SW:
                    continue
            for k in tokens2[i]:
                if SW is not None:
                    if k in SW:
                        continue
                n = max(len(k), len(j))
                l = Lev(k,j, substitution_cost=2)
                if l <= n/2:
                    tmp_dist += 1         
        d[i] = tmp_dist
        
    return d

471 2287 295


In [192]:
a,b =text2question_ratio(book1)

ld = levhenshtein_dist(a,b)
label = (book1['Book relevance'].values.reshape(-1,1)== 'Yes').astype(int)
df = pd.DataFrame(np.hstack((ld.reshape(-1,1),label)))

c = df.corr()
c

,0,1
0,1.000000,0.433395
1,0.433395,1.000000


In [281]:
import string
def prop_name_count(tokens, names):
    d = np.zeros(len(tokens))
    for i,M in enumerate(tokens):
        for j in M:
            if j.capitalize() in names:
                d[i] += 1
    return d

nms = pd.read_csv('../utils/Slovenska_Imena.txt', sep = ',', header= None).to_numpy()

prop_name_count(b, nms)


array([0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0.,
       0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 2., 0., 0., 1., 0., 0., 1., 0., 0., 1., 1.,
       0., 2., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 1.,
       1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 1., 1., 0., 0., 0., 0., 0.,
       0., 1., 0., 1., 0., 2., 1., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0.,
       1., 0., 0., 0., 0.